In [1]:
from generate import *
from sklearn.linear_model import *
from sklearn.svm import *
from sklearn.metrics import accuracy_score
from tqdm import *
from sklearn.externals import joblib

def generate_rollout_batch(filename, batch_size, last_n=None):
    actions = 0
    batch_samples = np.zeros((batch_size, 15*15*3), dtype='float32')  # TODO: refactor hard-code
    batch_labels = np.zeros((batch_size,), dtype='float32')
    index = 0
    while True:
        for game in generate_games(filename):
            for action_idx, action in enumerate(game.generate_actions(last_n=last_n)):
                batch_samples[index], batch_labels[index] = action[0].flatten(), action[1][0].argmax()
                index += 1
                index %= batch_size
                actions += 1
                if index == 0:
                    yield (batch_samples, batch_labels)
 
actions = {}
count = 0
for batch in generate_rollout_batch('aug_train.renju', 1):
    if batch[1][0] not in actions:
        actions[batch[1][0]] = batch[0][0]
        count += 1
    if count == 225:
        break
x, y = next(generate_rollout_batch('aug_train.renju', 65536))
x1, y1 = next(generate_rollout_batch('aug_train.renju', 32768, last_n=10))
x2, y2 = next(generate_rollout_batch('aug_train.renju', 8192, last_n=1))
x_test, y_test = next(generate_rollout_batch('aug_test.renju', 20000))
x = np.vstack((x1, x, x2, *[actions[act] for act in actions]))
y = np.hstack((y1, y, y2, np.array([act for act in actions])))
print('data generated')
sys.stdout.flush()

Using TensorFlow backend.


data generated


In [2]:
model = LogisticRegression(n_jobs=-1)
%time model.fit(x, y)
print('fitted')
print(accuracy_score(y_test, model.predict(x_test)))
joblib.dump(model, 'filtered_sklearn_policy.pkl')

CPU times: user 57min 51s, sys: 776 ms, total: 57min 52s
Wall time: 57min 52s
fitted
0.0651


['filtered_sklearn_policy.pkl']

In [ ]:
from sklearn.linear_model import *
from sklearn.externals import joblib
model = LogisticRegression()
model.fit(samples, labels)
joblib.dump(model, 'path_to_save')
loaded_model = joblib.load('path_to_model')
action_probas = loaded_model.predict_proba(array)

In [ ]:
board = np.zeros((15, 15), dtype='float32')
%timeit model.predict(prepare_board(board, True).flatten()[np.newaxis])

In [ ]:
from sklearn.externals import joblib

In [ ]:
joblib.dump(model, 'sklearn_policy.pkl')

In [ ]:
board = np.zeros((15, 15))
board[7, 7] = -1.0
board[8, 7] = 1.0
board[8, 8] = -1.0
show_board(model.predict_proba(prepare_board(board, True).flatten()[np.newaxis]).reshape(15, 15))